# Deep Learning 2

In dit practicum gaan we een diep neuraal netwerk trainen om kledingstukken te classificeren op basis van foto's. We gebruiken hiervoor de [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset, die reeds in Keras zit ingebouwd:

In [ ]:
import keras
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

Laten we eens kijken naar een paar samples.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print('Training set shape: {}'.format(x_train.shape))
print('Test set shape: {}'.format(x_test.shape))

sample = x_train[np.random.randint(x_train.shape[0])]
plt.imshow(sample, cmap='gray_r')
plt.axis('off')
plt.show()

Fashion-MNIST bestaat uit 70.000 zwart-witte afbeeldingen van 28x28 pixels groot, gemaakt op basis van foto's van kledij die verkocht wordt door [Zalando](https://www.zalando.be) (nee, ik word niet extra betaald voor dit product placement).

## Bouw een classifier

We gaan nu een model maken voor Fashion-MNIST:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='sgd',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

Keras kan ons een mooi overzicht geven van wat er allemaal gaande is in dit model:

In [ ]:
model.summary()

Zoals je ziet heeft dit model 421.576 vrije parameters. Dit lijkt misschien veel, maar moderne DNN's hebben makkelijk miljoenen parameters. Deze architectuur is echter wel al redelijk typisch voor hoe moderne DNN's werken voor beeldherkenning. Het bestaat uit afwisselende lagen van convolutionele en pooling operaties gevolgd door fully-connected lagen. De convolutionele lagen kunnen gezien worden als *feature extractors*: hun taak is leren relevante informatie te extraheren uit de input zodat de fully-connected lagen op het einde makkelijk een classificatie kunnen doen. Als je `pydot` en `graphviz` hebt staan, kan Keras het model zelfs visualiseren:

In [ ]:
from keras.utils import plot_model
from PIL import Image
plot_model(model, to_file='model.png')

im = Image.open('model.png')
im

Voordat we dit model kunnen trainen op Fashion-MNIST, moeten we nog een paar technische details afhandelen:

1. De labels moeten nog one-hot encoded worden.
2. De samples zijn niet genormaliseerd.
3. De samples moeten 4-dimensionale vectoren zijn om compatibel te worden met de convolutionele lagen.

We brengen dat allemaal in orde en trainen het model:

In [ ]:
from keras.utils import to_categorical

y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

x_mean = np.mean(x_train, axis=0)
x_std = np.std(x_train, axis=0)
x_train_4d = ((x_train - x_mean) / x_std).reshape(*x_train.shape, 1)
x_test_4d = ((x_test - x_mean) / x_std).reshape(*x_test.shape, 1)

history = model.fit(x_train_4d, y_train_oh,
          batch_size=50,
          epochs=10,
          verbose=1,
          validation_data=(x_test_4d, y_test_oh))

Dit proces kan wel even duren. Achteraf valideren we het resultaat:

In [ ]:
scores = model.evaluate(x_test_4d, y_test_oh, batch_size=50)
print('Loss: {}'.format(scores[0]))
print('Accuracy: {}'.format(scores[1]))

Je zou rond de 90% accuracy moeten kunnen halen.

Keras geeft ook een `History` object terug, waarmee we de geschiedenis van het trainen kunnen bekijken:

In [ ]:
plt.plot(history.history['acc'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

Op deze plot kan je de evolutie van het model zien tijdens de training. Typisch gedrag hier is een snelle stijging in het begin die na een tijdje afzwakt tot een lijn die fluctueert rond een gemiddelde waarde. In het geval van overfitting zal je de accuracy zien dalen naarmate je langer traint, maar dat zou hier niet het geval mogen zijn.

We kunnen nu afbeeldingen annoteren met een label:

In [ ]:
idx = np.random.randint(x_test.shape[0])
sample = x_test[idx]
test_sample = x_test_4d[idx]
index = np.argmax(model.predict(test_sample.reshape(1, *test_sample.shape)))
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

label = classes[index]
plt.title(label)
plt.imshow(sample, cmap='gray_r')
plt.axis('off')
plt.show()

## Oefening 1

Gegeven dat [MLP's eender welke functie kunnen benaderen](https://www.wikiwand.com/en/Universal_approximation_theorem), waarom houden we ons dan bezig met speciale operaties zoals convoluties en pooling?

## Oefening 2

Neurale netwerken horen robuust te zijn tegen allerhande soorten ruis. Test hoe robuust ons model is tegen uniforme en normale ruis:

$$\begin{aligned}
    v &= x + \varepsilon u,
    &
    w &= x + \varepsilon g,
\end{aligned}$$

waar $x$ een sample is uit de testset, $\varepsilon$ een getal is, $u$ uniforme ruis is in het interval $[-1,1]$ en $g$ standaardnormale ruis. Experimenteer met verschillende waarden van $\varepsilon$ en bepaal de accuracy voor elke waarde wanneer de test samples aangetast worden door de ruis. Je mag de experimenten beperken tot $\varepsilon \in [0,1]$ aangezien $\varepsilon=1$ al een aanzienlijke storing introduceert.

**Opmerking.** Het toevoegen van ruis kan ervoor zorgen dat de pixelwaarden niet meer in het geldige interval liggen waar het model op getraind is. [Clip de ruiswaarden](https://docs.scipy.org/doc/numpy/reference/generated/numpy.clip.html) om dit te voorkomen.

## Oefening 3

Keras verschaft ook toegang tot de functionaliteit van de backend (zie [de documentatie](https://keras.io/backend/)). We kunnen hiermee bijvoorbeeld gradiënten bepalen van het model voor bepaalde inputs:

In [ ]:
import keras.backend as K

sample = x_test_4d[np.random.randint(x_test_4d.shape[0])]
sample = sample.reshape(1, *sample.shape)
target_idx = model.predict(sample).argmax()
target = to_categorical(target_idx, 10)
target_variable = K.variable(target)
loss = keras.metrics.categorical_crossentropy(model.output, target_variable)
gradients = K.gradients(loss, model.input)
get_grad_values = K.function([model.input], gradients)
grad_values = get_grad_values([sample])[0]

De array `grad_values` bevat nu gradiënten van de verliesfunctie tov het `sample` dat we willekeurig uit de testset hebben gekozen. Wat gebeurt er als we een dergelijk sample nemen en een geschaalde versie van de tekens van de gradiënten eraan toevoegen? We berekenen dus

$$
    \tilde{x} = x + \varepsilon \cdot \mathrm{sgn}(g)
$$

met $x$ een test sample, $\varepsilon > 0$ en $g$ de gradiënten van de verliesfunctie in $x$. Visualiseer $x$ en $\tilde{x}$ voor een aantal samples en vergelijk de accuraatheid van de predicties van het model op de originele en gewijzigde samples. Welke conclusies kan je hieruit trekken?

## Oefening 4

De [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) bestaat uit 60.000 RGB kleurenafbeeldingen van 32x32 pixels groot. Probeer eens voor deze dataset een convolutioneel neuraal netwerk op te stellen dat een zo hoog mogelijke accuracy behaalt. De state-of-the-art ligt ten tijde van dit schrijven rond 98% (zie [deze paper](https://arxiv.org/abs/1805.09501)).

## Oefening 5

Convolutionele neurale netwerken werden oorspronkelijk voorgesteld voor beeldverwerking, maar in feite kan men ze toepassen op nog andere gebieden. Een voorbeeld hiervan zijn de *character-level language CNNs*: CNNs die gemaakt zijn voor tekstverwerking. Indien de lengte van de tekst begrensd is, kan men perfect een gewoon feedforward CNN hierop toepassen; een recurrent netwerk is enkel nodig indien de reeksen variabele lengte hebben.

Keras heeft ook de IMDB movie review dataset ingebouwd. Deze dataset bevat reviews van films (geëncodeerd als lijsten van integers) samen met een label (positief of negatief). Bouw een CNN voor classificatie van deze movie reviews. **Hint:** kijk eens naar de [Embedding layer](https://keras.io/layers/embeddings/) van Keras.

Als je eens wil spelen met recurrente netwerken, kan je het CNN van deze oefening ook eens vergelijken met een RNN. Keras heeft hier [uitgebreide ondersteuning](https://keras.io/layers/recurrent/) voor. Normaal gezien zou een LSTM geschikt moeten zijn voor deze taak, maar voel je vrij om te experimenteren.